### Import libraries and dependencies

In [1]:
import pandas as pd
import pycountry
import re
from functools import reduce

import warnings
warnings.filterwarnings("ignore")

pd.options.display.float_format = '{:,.0f}'.format

In [2]:
def excel_parser(file_path):
    '''
    This function takes the file path and read multiple sheets in it while parsing them through the necessary 
    removal from the excel cells. Then it writes each sheet as a dataframe and collects it in dictionary.
    '''
    xl = pd.ExcelFile(file_path)
    xl_sheets = xl.sheet_names[1:-2]

    all_sheets_df = []
    for sheet in xl_sheets:
        sheets_dict = {}
        df_name = 'df_'+sheet.replace(" ","")
        
        df = xl.parse(sheet, skiprows=14, convert_float=False, header=None)
        
        for i in df.iloc[:1].columns:
            if type(df.iloc[:1][i][0])==str:
                
                # replace "International migrant stock at mid-year by age (both sexes)" by 'both_sexes' in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock at mid-year by age (both sexes)','both_sexes')
                
                # replace "International migrant stock at mid-year by age " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock at mid-year by age ','')
                
                # replace "Total population of both sexes at mid-year (thousands)g" by "both_sexes" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Total population of both sexes at mid-year (thousands)g','both_sexes')
                
                # replace "Total female population at mid-year (thousands)" by "(female)" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Total female population at mid-year (thousands)','female')
                
                # replace "Total male population at mid-year (thousands)" by "(male)" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Total male population at mid-year (thousands)','male')
                
                # replace "International migrant stock as percentage of the total population " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock as percentage of the total population ','')
                
                # replace "Percentage distribution of the international migrant stock " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Percentage distribution of the international migrant stock ','')
                
                # replace "Female migrants as a percentage of the international migrant stock" by "FemaleMigr_%_IMS" in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Female migrants as a percentage of the international migrant stock','FemaleMigr_%_IMS')
                
                # replace "International migrant stock at mid-year " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('International migrant stock at mid-year ','')

                # replace "Annual rate of change of the migrant stock " by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Annual rate of change of the migrant stock ','')

                # replace "Estimated refugee stock (including asylum seekers) at mid-year (both sexes)" by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Estimated refugee stock (including asylum seekers) at mid-year (both sexes)','both_sexes')

                # replace "Refugees and asylum seekers as a percentage of the international migrant stock" by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Refugees and asylum seekers as a percentage of the international migrant stock','RefAsylum_%_IMS')

                # replace "Annual rate of change of the refugee stock (including asylum seekers)" by empty in header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('Annual rate of change of the refugee stock (including asylum seekers)','ARchange_ofRefAsylum')

                # remove "(", ")" from the row that is going to be header
                df.iloc[:1][i][0] = df.iloc[:1][i][0].replace('(','').replace(')','')

        #Then first forward fill the NaNs in the first two rows (thus propagating 'Auto loan', for example).
        df.iloc[0:2] = df.iloc[0:2].fillna(method='ffill', axis=1)
        #Next fill in the remaining NaNs with empty strings:
        df.iloc[0:2] = df.iloc[0:2].fillna('')

        #Now join the two rows together with '_' and assign that as the column level values:
        df.columns = df.iloc[0:2].apply(lambda x: '_'.join([str(y) for y in x if y]), axis=0)

        #And finally, remove the first two rows:
        df = df.iloc[2:]
        
        #Remove the rows woth all NaN values
        df = df.dropna(how='all')
        
        #Convert year from float to integer
        if df.columns.str.contains("Year").any()==True:
            df['Year'] = df['Year'].astype(int)
        
        sheets_dict = {df_name: df}
        all_sheets_df.append(sheets_dict)
    return(all_sheets_df)

## I. Read Migrant Stock Datasets

### i. Reading dataset of _UN Migrant Stock By Age And Sex 2019_

In [3]:
#Read the file
file_path_AgeSex = "./data/UN_data/UN_MigrantStockByAgeAndSex_2019.xlsx"
MigrStock_ageSex = excel_parser(file_path_AgeSex)

df_Table1_AgeSex = MigrStock_ageSex[0]['df_Table1'].reset_index(drop=True)
df_Table2_AgeSex = MigrStock_ageSex[1]['df_Table2'].reset_index(drop=True)
# df_Table3_AgeSex = MigrStock_ageSex[2]['df_Table3'].reset_index(drop=True)
# df_Table4_AgeSex = MigrStock_ageSex[3]['df_Table4'].reset_index(drop=True)
# df_Table5_AgeSex = MigrStock_ageSex[4]['df_Table5'].reset_index(drop=True)

df_ANNEX_AgeSex = pd.ExcelFile(file_path_AgeSex).parse('ANNEX', skiprows=15, convert_float=False)\
                                                .drop(['Index'], axis=1)


#### Create a dataframe of countries' development info


In [11]:
# Create a dataframe of countries' development info
countries_development_info = df_ANNEX_AgeSex[df_ANNEX_AgeSex['Type of data'].notnull()]
countries_development_info = countries_development_info.drop(['Notes', 'Type of data'], axis=1)
countries_development_info = countries_development_info.fillna('N')

In [12]:
# keep only one detailed info
countries_development_info.loc[(countries_development_info['Less Developed Regions']=="Y") & (countries_development_info['Least developed countries']=="Y"),'Less Developed Regions'] = 'N'

countries_development_info.loc[((countries_development_info['Upper-middle-income Countries']=="Y") & (countries_development_info['Middle-income Countries']=="Y")) | ((countries_development_info['Lower-middle-income Countries']=="Y") & (countries_development_info['Middle-income Countries']=="Y")),'Middle-income Countries'] = 'N'



In [14]:
# Pivot all the development info columns to rows with new variable 'DevelopmentLevel' and 'Yes_No'
countries_development_info_melted = countries_development_info\
                                    .melt(id_vars=['Region, subregion, country or area', 'Code'], 
                                          var_name="DevelopmentLevel", 
                                          value_name="YesNo")\
                                    .reset_index(drop=True)

In [15]:
set(countries_development_info_melted.DevelopmentLevel)

{'High-income Countries',
 'Least developed countries',
 'Less Developed Regions',
 'Low-income Countries',
 'Lower-middle-income Countries',
 'Middle-income Countries',
 'More Developed Regions',
 'No income group available',
 'Sub-Saharan Africa',
 'Upper-middle-income Countries'}

In [16]:
# dataframe shape corrections
countries_development_info_melted['Code'] = countries_development_info_melted['Code'].astype(int)
countries_development_info_melted.rename(columns={'Region, subregion, country or area': 'Region'}, inplace=True)

In [17]:
countries_development_info_melted.head()

,Region,Code,DevelopmentLevel,YesNo
0,Burundi,108,More Developed Regions,N
1,Comoros,174,More Developed Regions,N
2,Djibouti,262,More Developed Regions,N
3,Eritrea,232,More Developed Regions,N
4,Ethiopia,231,More Developed Regions,N


In [18]:
# Save to a csv
countries_development_info_melted.to_csv('./csv/countries_development_info.csv', encoding='utf-8', index=False)

#### Preparing datasets for interactive plots purpose

In [10]:
df1 = df_Table1_AgeSex[df_Table1_AgeSex['Type of data a'].notnull()]\
                .melt(id_vars=['Year', 'Sort\norder', 'Major area, region, country or area of destination', 
                               'Notes', 'Code', 'Type of data a'], 
                      var_name="GenderAge",
                      value_name="InternationalMigrantStocks")\
                .drop(['Notes', 'Type of data a'], axis=1)\
                .reset_index(drop=True)

df1[['Gender', 'AgeGroup']] = pd.DataFrame([i[1:3] for i in df1.GenderAge.str.split(r"^([^\d]+)")], 
                                         columns = ['Gender', 'AgeGroup'])

df1.Gender = [i.replace("_", "") for i in df1.Gender]
df1.AgeGroup = [i.replace("-", "_") for i in df1.AgeGroup]

total_infos = ['bothsexesTotal', 'maleTotal', 'femaleTotal']
df1 = df1[~df1['Gender'].isin(total_infos)]
df1 = df1.drop(['GenderAge'], axis=1).reset_index(drop=True)

In [11]:
df2 = df_Table2_AgeSex[df_Table2_AgeSex['Type of data a'].notnull()]\
                .melt(id_vars=['Year', 'Sort\norder', 'Major area, region, country or area of destination', 
                               'Notes', 'Code', 'Type of data a'], 
                      var_name="GenderAge",
                      value_name="TotalPopulation")\
                .drop(['Notes', 'Type of data a'], axis=1)\
                .reset_index(drop=True)

df2[['Gender', 'AgeGroup']] = pd.DataFrame([i[1:3] for i in df2.GenderAge.str.split(r"^([^\d]+)")], 
                                         columns = ['Gender', 'AgeGroup'])

df2.Gender = [i.replace("_", "") for i in df2.Gender]
df2.AgeGroup = [i.replace("-", "_") for i in df2.AgeGroup]

# since total population is given in thousands we multiplied with 1000
df2.TotalPopulation = [i*1000 for i in df2.TotalPopulation] 

total_infos = ['bothsexesTotal', 'maleTotal', 'femaleTotal']
df2 = df2[~df2['Gender'].isin(total_infos)]
df2 = df2.drop(['GenderAge'], axis=1).reset_index(drop=True)

In [12]:
# df3 = df_Table3_AgeSex[df_Table3_AgeSex['Type of data a'].notnull()]\
#                 .melt(id_vars=['Year', 'Sort\norder', 'Major area, region, country or area of destination', 
#                                'Notes', 'Code', 'Type of data a'], 
#                       var_name="GenderAge",
#                       value_name="InternationalMigrantStockPercentage")\
#                 .drop(['Notes', 'Type of data a'], axis=1)\
#                 .reset_index(drop=True)

# df3[['Gender', 'AgeGroup']] = pd.DataFrame([i[1:3] for i in df3.GenderAge.str.split(r"^([^\d]+)")], 
#                                          columns = ['Gender', 'AgeGroup'])

# df3.Gender = [i.replace(" ", "").replace("_", "") for i in df3.Gender]
# df3.AgeGroup = [i.replace("-", "_") for i in df3.AgeGroup]

# total_infos = ['bothsexesTotal', 'maleTotal', 'femaleTotal']
# df3 = df3[~df3['Gender'].isin(total_infos)]
# df3 = df3.drop(['GenderAge'], axis=1).reset_index(drop=True)

In [13]:
df1.set_index(['Year', 'Sort\norder', 'Code',  
               'Major area, region, country or area of destination', 'Gender', 'AgeGroup'],  inplace=True)

df2.set_index(['Year', 'Sort\norder', 'Code',  
               'Major area, region, country or area of destination', 'Gender', 'AgeGroup'],  inplace=True)

# merge dataframes
merged1 = pd.merge(df1, df2, left_index=True, right_index=True)

In [14]:
# drop unknown International Migrant Stocks and Total Population Numbers
unknown_stock_pop = ["..", 
                     ".................................................", 
                     "................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................"]
merged1 = merged1[~merged1['InternationalMigrantStocks'].isin(unknown_stock_pop)]
merged1 = merged1[~merged1['TotalPopulation'].isin(unknown_stock_pop)]
merged1['IMSPercentageofTotalPop'] = (merged1.InternationalMigrantStocks/merged1.TotalPopulation).map(lambda n: '{:,.1%}'.format(n))

# dataframe shape corrections
merged1.reset_index(inplace=True)
merged1['Code'] = merged1['Code'].astype(int)
merged1['InternationalMigrantStocks'] = merged1['InternationalMigrantStocks'].astype(int)
merged1['TotalPopulation'] = merged1['TotalPopulation'].astype(int)
merged1.rename(columns={'Major area, region, country or area of destination': 'Destination'}, inplace=True)
merged1 = merged1.drop(['Sort\norder'], axis=1).reset_index(drop=True)

In [15]:
merged1.head()

,Year,Code,Destination,Gender,AgeGroup,InternationalMigrantStocks,TotalPopulation,IMSPercentageofTotalPop
0,1990,108,Burundi,bothsexes,0_4,22459,1096535,2.0%
1,1990,174,Comoros,bothsexes,0_4,600,75107,0.8%
2,1990,262,Djibouti,bothsexes,0_4,5415,107065,5.1%
3,1990,232,Eritrea,bothsexes,0_4,718,403753,0.2%
4,1990,231,Ethiopia,bothsexes,0_4,77249,9056989,0.9%


In [16]:
# Save to a csv
merged1.to_csv('./csv/International_and_totalStock_GenderAge.csv', encoding='utf-8', index=False)

### ii. Reading dataset of _UN Migrant Stock By Origin And Destination 2019_

In [17]:
#Read the file
file_path_OriginDestination = "./data/UN_data/UN_MigrantStockByOriginAndDestination_2019.xlsx"
MigrStock_originDestination = excel_parser(file_path_OriginDestination)

df_Table1_OriginDest = MigrStock_originDestination[0]['df_Table1'].reset_index(drop=True)
df_Table2_OriginDest = MigrStock_originDestination[1]['df_Table2'].reset_index(drop=True)
df_Table3_OriginDest = MigrStock_originDestination[2]['df_Table3'].reset_index(drop=True)

# df_ANNEX_OriginDest = pd.ExcelFile(file_path_OriginDestination).parse('ANNEX', skiprows=15, convert_float=False).drop(['Index'], axis=1)


#### Preparing datasets for interactive plots purpose

In [18]:
df1_o = df_Table1_OriginDest[df_Table1_OriginDest['Type of data a'].notnull()]\
                .melt(id_vars=['Year', 'Sort\norder', 'Major area, region, country or area of destination', 
                               'Notes', 'Code', 'Type of data a'], 
                      var_name="Origin",
                      value_name="TotalMigrantStocks")\
                .drop(['Notes', 'Type of data a'], axis=1)\
                .reset_index(drop=True)

df1_o['Country or area of origin'] = pd.DataFrame([i[1] for i in df1_o.Origin.str.split("_")], 
                                                  columns = ['Country or area of origin'])

total_infos = ['Total']
df1_o = df1_o[~df1_o['Country or area of origin'].isin(total_infos)]
df1_o = df1_o.drop(['Origin'], axis=1).reset_index(drop=True)

In [19]:
df2_o = df_Table2_OriginDest[df_Table2_OriginDest['Type of data a'].notnull()]\
                .melt(id_vars=['Year', 'Sort\norder', 'Major area, region, country or area of destination', 
                               'Notes', 'Code', 'Type of data a'], 
                      var_name="Origin",
                      value_name="MaleMigrantStocks")\
                .drop(['Notes', 'Type of data a'], axis=1)\
                .reset_index(drop=True)

df2_o['Country or area of origin'] = pd.DataFrame([i[1] for i in df2_o.Origin.str.split("_")], 
                                                  columns = ['Country or area of origin'])

total_infos = ['Total']
df2_o = df2_o[~df2_o['Country or area of origin'].isin(total_infos)]
df2_o = df2_o.drop(['Origin'], axis=1).reset_index(drop=True)

In [20]:
df3_o = df_Table3_OriginDest[df_Table3_OriginDest['Type of data a'].notnull()]\
                .melt(id_vars=['Year', 'Sort\norder', 'Major area, region, country or area of destination', 
                               'Notes', 'Code', 'Type of data a'], 
                      var_name="Origin",
                      value_name="FemaleMigrantStocks")\
                .drop(['Notes', 'Type of data a'], axis=1)\
                .reset_index(drop=True)

df3_o['Country or area of origin'] = pd.DataFrame([i[1] for i in df3_o.Origin.str.split("_")], 
                                                  columns = ['Country or area of origin'])

total_infos = ['Total']
df3_o = df3_o[~df3_o['Country or area of origin'].isin(total_infos)]
df3_o = df3_o.drop(['Origin'], axis=1).reset_index(drop=True)

In [21]:
df1_o.set_index(['Year', 'Sort\norder', 'Code', 'Major area, region, country or area of destination', 
                'Country or area of origin'],  inplace=True)

df2_o.set_index(['Year', 'Sort\norder', 'Code', 'Major area, region, country or area of destination', 
                'Country or area of origin'],  inplace=True)

df3_o.set_index(['Year', 'Sort\norder', 'Code', 'Major area, region, country or area of destination', 
                'Country or area of origin'],  inplace=True)

# merge dataframes
dfs = [df1_o, df2_o, df3_o]
df_final = reduce(lambda left,right: pd.merge(left, right, left_index=True, right_index=True), dfs)

In [22]:
# drop unknown Migrant Stocks Numbers
unknown_stock_pop = ["..", "-"]
df_final = df_final[~df_final['TotalMigrantStocks'].isin(unknown_stock_pop)]
df_final = df_final[~df_final['MaleMigrantStocks'].isin(unknown_stock_pop)]
df_final = df_final[~df_final['FemaleMigrantStocks'].isin(unknown_stock_pop)]

# drop rows with all NaNs
df_final = df_final.dropna(how='all')

# remove NaNs from TotalMigrantStocks 
df_final = df_final[df_final['TotalMigrantStocks'].notna()]

# Drop all zero rows for TotalMigrantStocks
zeroStock_subset = df_final[df_final.TotalMigrantStocks==0]
df_final = df_final.drop(zeroStock_subset.index)

# finally fill NaN values with zero since df_final['TotalMigrantStocks']-df_final['MaleMigrantStocks']=df_final['FemaleMigrantStocks']
df_final = df_final.fillna(0)

In [23]:
# dataframe shape corrections
df_final.reset_index(inplace=True)
df_final['Code'] = df_final['Code'].astype(int)
df_final['TotalMigrantStocks'] = df_final['TotalMigrantStocks'].astype(int)
df_final['MaleMigrantStocks'] = df_final['MaleMigrantStocks'].astype(int)
df_final['FemaleMigrantStocks'] = df_final['FemaleMigrantStocks'].astype(int)
df_final.rename(columns={'Major area, region, country or area of destination': 'Destination'}, inplace=True)
df_final = df_final.drop(['Sort\norder'], axis=1).reset_index(drop=True)

In [24]:
df_final.head()

,Year,Code,Destination,Country or area of origin,TotalMigrantStocks,MaleMigrantStocks,FemaleMigrantStocks
0,1990,108,Burundi,Other South,50676,24837,25839
1,1990,174,Comoros,Other South,847,432,415
2,1990,262,Djibouti,Other South,5484,3056,2428
3,1990,232,Eritrea,Other South,737,390,347
4,1990,231,Ethiopia,Other South,22075,11603,10472


In [25]:
# Save to a csv
df_final.to_csv('./csv/GenderBasedMigrationStock.csv', encoding='utf-8', index=False)

### iii. Reading dataset of _UN Migrant Stock Total 2019_

It seems we don't need to use this dataset.

In [26]:
# #Read the file
# file_path_StockTotal = "./data/UN_data/UN_MigrantStockTotal_2019.xlsx"
# MigrStock_StockTotal = excel_parser(file_path_StockTotal)

# df_Table1_StockTotal = MigrStock_StockTotal[0]['df_Table1'].reset_index(drop=True)
# df_Table2_StockTotal = MigrStock_StockTotal[1]['df_Table2'].reset_index(drop=True)
# df_Table3_StockTotal = MigrStock_StockTotal[2]['df_Table3'].reset_index(drop=True)
# df_Table4_StockTotal = MigrStock_StockTotal[3]['df_Table4'].reset_index(drop=True)
# df_Table5_StockTotal = MigrStock_StockTotal[4]['df_Table5'].reset_index(drop=True)
# df_Table6_StockTotal = MigrStock_StockTotal[5]['df_Table6'].reset_index(drop=True)

# df_ANNEX_StockTotal = pd.ExcelFile(file_path_StockTotal).parse('ANNEX', skiprows=15, convert_float=False).drop(['Index'], axis=1)


## II. Read Migrant Flow Datasets

### i. migflows_allcountries_1990_2015.csv Data Details:

**Row for each migration corridor - period combination (200 origins x 200 destinations x 5 periods = 200,000).**

- year0 - first year of five year period
- orig - origin ISO three letter country code
- dest - destination ISO three letter country

**Columns for estimates based on the following migration flow estimation methods:**

Stock Differencing Approaches: 
- sd_drop_neg - https://doi.org/10.1016/j.jdeveco.2009.11.004
- sd_rev_neg - https://doi.org/10.1111/sjoe.12098

Migration Rate Approach:
- mig_rate - https://doi.org/10.1002/9781118937464.ch7]

Demographic Accounting Approaches:
- da_min_open - https://doi.org/10.4054/DemRes.2013.28.18
- da_min_closed - https://doi.org/10.1111/imre.12327
- da_pb_closed - https://doi.org/10.1073/PNAS.1722334116

In [27]:
# Read the file
df_MigrFlows = pd.read_csv("./data/Abel/migflows_allcountries_1990_2015.csv")
df_MigrFlows.head()

,year0,orig,dest,sd_drop_neg,sd_rev_neg,mig_rate,da_min_open,da_min_closed,da_pb_closed
0,1990,BDI,BDI,0,0,0,0,0,0
1,1990,COM,BDI,0,0,0,0,0,0
2,1990,DJI,BDI,0,0,0,0,0,0
3,1990,ERI,BDI,0,0,0,0,0,83
4,1990,ETH,BDI,0,0,0,0,0,2


### ii. migflows_allcountries_gender_separated_1990_2015.csv Data Details

**Column details:**
- stock: stock data source used for the estimated flow (un12, un13, un15 or wb11)
- demo: demographic data source used for the estimated flow (wpp2010, wpp2012, wpp2015)
- sex: gender of the estimated flow
- year0: first year of the period of the estimated flow (ranging from 1960 to 2010, and varying in lengths depending on the stock and demo data, as described in the paper)
- interval: the length of the period of the estimated flow 
- orig: ISO 3166-1 alpha-3 letter code for the origin country of the estimated flow
- dest: ISO 3166-1 alpha-3 letter code for the destination country of the estimated flow
- orig_code: ISO 3166-1 numeric code for the origin country of the estimated flow
- dest_code: ISO 3166-1 numeric code for the destination country of the estimated flow
- flow: estimated flow

More details on the country codes:
- https://www.iso.org/iso-3166-country-codes.html
- https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3
- https://en.wikipedia.org/wiki/ISO_3166-1_numeric

More details on tidy data:
- http://vita.had.co.nz/papers/tidy-data.pdf

In [28]:
# Read the file
df_MigrFlowSex = pd.read_csv("./data/Abel/migflows_allcountries_gender_separated_1990_2015.csv")

In [29]:
# take the most recent estimations done
df_MigrFlowSex = df_MigrFlowSex[(df_MigrFlowSex.stock=='un15') & (df_MigrFlowSex.demo=='wpp2015')]

# take the flow values with 5-year interval
df_MigrFlowSex = df_MigrFlowSex[df_MigrFlowSex.interval==5]

# drop the zero flow values
df_MigrFlowSex = df_MigrFlowSex[df_MigrFlowSex.flow!=0]

# now drop unnecessary columns 
df_MigrFlowSex = df_MigrFlowSex.drop(['stock', 'demo', 'interval'], axis=1).reset_index(drop=True)
df_MigrFlowSex.head()

,sex,year0,orig,dest,orig_code,dest_code,flow
0,b,1990,ABW,CAN,533,124,1
1,b,1990,ABW,DEU,533,276,9
2,b,1990,ABW,FRA,533,250,1
3,b,1990,ABW,GBR,533,826,1
4,b,1990,ABW,NLD,533,528,1


## III. Read Refugee population by country or territory of asylum Datasets

Useful information about some terms in the Refugee-Asylum Dataset:

- Refugees are people who are recognized as refugees under the 1951 Convention Relating to the Status of Refugees or its 1967 Protocol, the 1969 Organization of African Unity Convention Governing the Specific Aspects of Refugee Problems in Africa, people recognized as refugees in accordance with the UNHCR statute, people granted refugee-like humanitarian status, and people provided temporary protection. 

- Asylum seekers--people who have applied for asylum or refugee status and who have not yet received a decision or who are registered as asylum seekers--are excluded. 

- Palestinian refugees are people (and their descendants) whose residence was Palestine between June 1946 and May 1948 and who lost their homes and means of livelihood as a result of the 1948 Arab-Israeli conflict. Country of asylum is the country where an asylum claim was filed and granted.


__Source:__ Data before 2018 are from United Nations High Commissioner for Refugees ( UNHCR ), Statistics Database, Statistical Yearbook and data files, complemented by statistics on Palestinian refugees under the mandate of the UNRWA as published on its website. Data from UNHCR are available online at: unhcr.org/en-us/figures-at-a-glance.html. The data for 2018 are from UNHCR Global Trends report 2018 and World Bank's estimates ( for Palestinian refugees in the UNRWA areas of operation ).


In [30]:
# Read the file
file_path_Refugee = "./data/TheWorldBankRefugee/API_SM.POP.REFG_DS2_en_excel_v2_890428.xls"
df_Refugee = pd.ExcelFile(file_path_Refugee).parse('Data', skiprows=3)

In [31]:
# Remove all the NaN columns that have no value for the further analysis from the data 
# find all the columns which have all NaN values in it and drop unnecessary columns
nan_columns = df_Refugee.columns[df_Refugee.isnull().all()]#.tolist()
df_Refugee = df_Refugee.drop(nan_columns, axis=1)
df_Refugee = df_Refugee.drop(['Indicator Name', 'Indicator Code'], axis=1)
df_Refugee = df_Refugee.fillna(0, axis=1)
df_Refugee.head()

,Country Name,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,2,1,0,0
1,Afghanistan,AFG,50,38,"60,025","32,132","19,131","19,605","18,775",5,...,37,"6,434","3,009","16,187","16,863","300,423","257,554","59,771","75,927","72,231"
2,Angola,AGO,"11,557","11,022","11,002","10,878","10,686","10,884","9,381","9,364",...,"14,734","15,155","16,223","23,413","23,783","15,474","15,555","15,555","41,109","39,865"
3,Albania,ALB,0,0,"3,000","3,000","3,000","4,720","4,925",30,...,70,76,82,86,93,104,104,138,89,131
4,Andorra,AND,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
countries = set(df_Refugee['Country Code'].unique())
iso_countries = {country.alpha_3 for country in pycountry.countries}
not_iso_countries = countries - iso_countries
df_Refugee = df_Refugee[~df_Refugee['Country Code'].isin(not_iso_countries)]

In [33]:
# Drop 0 total subsets
df_Refugee['total'] = df_Refugee.iloc[:,2:-1].sum(axis=1)
zeroFlow_subset = df_Refugee[df_Refugee['total']==0]
df_Refugee = df_Refugee.drop(zeroFlow_subset.index).drop(['total'], axis=1).reset_index(drop=True)

In [34]:
# Pivot all the year columns to rows with new variable 'Year' and 'RefugeePopulation'
df_Refugee_map = df_Refugee.melt(id_vars=['Country Name', 'Country Code'], 
                                 var_name="Year", 
                                 value_name="RefugeePopulation")\
                            .rename(columns={'Country Name': 'CountryName', 'Country Code': 'CountryCode'})\
                            .reset_index(drop=True)
# Drop 0 RefugeePopulation subsets
zeroRefPop_subset = df_Refugee_map[df_Refugee_map['RefugeePopulation']==0]
df_Refugee_map = df_Refugee_map.drop(zeroRefPop_subset.index).reset_index(drop=True)
df_Refugee_map.head()

,CountryName,CountryCode,Year,RefugeePopulation
0,Afghanistan,AFG,1990,50
1,Angola,AGO,1990,"11,557"
2,Argentina,ARG,1990,"11,735"
3,Australia,AUS,1990,"97,915"
4,Austria,AUT,1990,"34,938"


In [35]:
# Save to a csv
df_Refugee_map.to_csv('./csv/RefugeePopulation.csv', encoding='utf-8', index=False)

## IV. Read Refugees and migrant deaths while trying to reach Europe

This dataset contains a list of events from 2000 to present during which someone died or went missing while trying to reach or stay in Europe. Details such as the date of the incident, the cause of death, number of dead and missing, location and coordinates of event are included.
[source](https://data.humdata.org/dataset/refugee-and-migrant-deaths-while-trying-to-reach-europe)

In [36]:
# Read the file
file_path_RefugeeMigrant_Deaths = "./data/migrantfiles_casualties/migrantfiles_casualties.xlsx"
df_RefugeeMigrant_Deaths = pd.ExcelFile(file_path_RefugeeMigrant_Deaths).parse('Events during which someone die')

In [37]:
# Drop unnecessary columns
df_RefugeeMigrant_Deaths = df_RefugeeMigrant_Deaths.drop(['Event_id', 'dataset', 'date', 'Somme Dedoublement', 'name', 'source', 'source_url'], axis=1)
df_RefugeeMigrant_Deaths = df_RefugeeMigrant_Deaths.rename(columns={'route (Frontex)': 'route'})

In [38]:
# Drop all nan rows
nan_subset = df_RefugeeMigrant_Deaths[df_RefugeeMigrant_Deaths.isnull().all(axis=1)]
df_RefugeeMigrant_Deaths = df_RefugeeMigrant_Deaths.drop(nan_subset.index).reset_index(drop=True)
# Fill nan values in 'route' columns with 'Unknown'
df_RefugeeMigrant_Deaths.route =  df_RefugeeMigrant_Deaths.route.fillna('Unknown')

In [39]:
# Save to a csv
df_RefugeeMigrant_Deaths.to_csv('./csv/RefugeeMigrant_Deaths.csv', encoding='utf-8', index=False)